In [29]:
#Victor de Lima Souza
#199335

In [30]:
import pandas as pd
import sqlite3

def load_to_sql_in_chunks(filepath, tablename, conn, chunksize=100000):
    """
    Lê um arquivo TSV.GZ em blocos e grava no banco SQLite.
    """
    print(f"🔄 Carregando {tablename} ...")
    chunks = pd.read_csv(filepath, sep='\t', compression='gzip', na_values='\\N', chunksize=chunksize)
    for i, chunk in enumerate(chunks):
        chunk.to_sql(tablename, conn, if_exists='append', index=False)
        print(f"   Inserido chunk {i+1}")

In [31]:
conn = sqlite3.connect("imdb.db")

In [32]:
load_to_sql_in_chunks("title.basics0.tsv.gz", "basics", conn)
load_to_sql_in_chunks("title.ratings.tsv.gz", "ratings", conn)
load_to_sql_in_chunks("title.principals0.tsv.gz", "principals", conn)

🔄 Carregando basics ...
   Inserido chunk 1
   Inserido chunk 2
   Inserido chunk 3
   Inserido chunk 4
   Inserido chunk 5
   Inserido chunk 6
   Inserido chunk 7
   Inserido chunk 8
   Inserido chunk 9
   Inserido chunk 10
   Inserido chunk 11
   Inserido chunk 12
   Inserido chunk 13
   Inserido chunk 14
   Inserido chunk 15
   Inserido chunk 16
   Inserido chunk 17
   Inserido chunk 18
   Inserido chunk 19
   Inserido chunk 20
   Inserido chunk 21
   Inserido chunk 22
   Inserido chunk 23
   Inserido chunk 24
   Inserido chunk 25
   Inserido chunk 26
   Inserido chunk 27
   Inserido chunk 28
   Inserido chunk 29
   Inserido chunk 30
   Inserido chunk 31
   Inserido chunk 32
   Inserido chunk 33
   Inserido chunk 34
   Inserido chunk 35
   Inserido chunk 36
   Inserido chunk 37
   Inserido chunk 38
   Inserido chunk 39
   Inserido chunk 40
   Inserido chunk 41
   Inserido chunk 42
   Inserido chunk 43
   Inserido chunk 44
   Inserido chunk 45
   Inserido chunk 46
   Inserido chunk 4

In [33]:
# Quais são os 5 filmes com as maiores notas?
query1 = """
SELECT b.primaryTitle, r.averageRating, r.numVotes
FROM ratings r
JOIN basics b ON r.tconst = b.tconst
WHERE b.titleType = 'movie'
ORDER BY r.averageRating DESC, r.numVotes DESC
LIMIT 5;
"""
top5 = pd.read_sql_query(query1, conn)
display(top5)

,primaryTitle,averageRating,numVotes
0,Kaveri,10.0,1023
1,Kurukku,10.0,451
2,Jedal Dar Omghe 30 Metri,10.0,142
3,Sargashte,10.0,134
4,Gorgeous Rascal,10.0,115


In [34]:
# Qual é o gênero mais frequente entre os filmes com nota maior que 8?
query2 = """
SELECT b.genres, COUNT(*) AS freq
FROM ratings r
JOIN basics b ON r.tconst = b.tconst
WHERE r.averageRating > 8 AND b.titleType = 'movie' AND b.genres IS NOT NULL
GROUP BY b.genres
ORDER BY freq DESC
LIMIT 1;
"""
genre_top = pd.read_sql_query(query2, conn)
display(genre_top)

,genres,freq
0,Documentary,7206


In [35]:
# Quais são os 3 atores/atrizes que mais participaram de filmes com nota maior que 7.5?
query3 = """
SELECT p.nconst, COUNT(*) AS qtd_filmes
FROM principals p
JOIN ratings r ON p.tconst = r.tconst
JOIN basics b ON b.tconst = p.tconst
WHERE r.averageRating > 7.5 AND b.titleType = 'movie' AND p.category IN ('actor', 'actress')
GROUP BY p.nconst
ORDER BY qtd_filmes DESC
LIMIT 3;
"""
top_actors = pd.read_sql_query(query3, conn)
display(top_actors)

,nconst,qtd_filmes
0,nm0004660,231
1,nm0595934,155
2,nm3183374,124


In [36]:
conn.close()